# Lecture 2: Memory

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

- **Large Language Models** never remember previous conversions

![memory](../images/L2/memory.png)

- **chatbots** store the previous conversations and give the whole
  convesation as context to **LLM**

![meomory with context](../images/L2/memory_with_context.png)

- `langchain` provides several kinds of memory to store and accumulate
   the conversation

## Types of memories

![Types of memories](../images/L2/memory_types.png)

## 1. ConversationBufferMemory

- Stores the entire conversation
- The whole conversation is given as a context to the next prompt

### Example 1.1: A chatbot using `langchain`'s `ConversationBufferMemory`

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [ ]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Hi, my name is Andrew")

"""
> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is \
talkative and provides lots of specific details from its context. If the AI \
does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.
"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"
"""

In [ ]:
conversation.predict(input="What is 1+1?")

"""
> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. \
The AI is talkative and provides lots of specific details from its \
context. If the AI does not know the answer to a question, it \
truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.
'The answer to 1+1 is 2.'
"""

In [ ]:
conversation.predict(input="What is my name?")

"""
> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. \
The AI is talkative and provides lots of specific details from \
its context. If the AI does not know the answer to a question, \
it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.
'Your name is Andrew, as you mentioned earlier.'
"""

In [ ]:
print(memory.buffer)

"""
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI: Your name is Andrew, as you mentioned earlier.
"""

In [ ]:
memory.load_memory_variables({})

"""
{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew, \
            it's nice to meet you. My name is AI. How can I \
            assist you today?\nHuman: What is 1+1?\nAI: The \
            answer to 1+1 is 2.\nHuman: What is my name?\nAI: \
            Your name is Andrew, as you mentioned earlier."
}
"""

In [ ]:
memory = ConversationBufferMemory()

In [ ]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [ ]:
print(memory.buffer)

"""
Human: Hi
AI: What's up
"""

In [ ]:
memory.load_memory_variables({})

"""
{'history': "Human: Hi\nAI: What's up"}
"""

In [ ]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [ ]:
memory.load_memory_variables({})

"""
{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}
"""

## 2. ConversationBufferWindowMemory

- This will only keep a window of memory

- the size of the window can be set by the user

### Example 2.1: A chatbot using `langchain`'s `ConversationBufferWindowMemory`

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(k=1)               

In [ ]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [ ]:
memory.load_memory_variables({})

# Note: only the last conversion is remembered

"""
{'history': 'Human: Not much, just hanging\nAI: Cool'}
"""

In [ ]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="Hi, my name is Andrew")

"""
> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. \
The AI is talkative and provides lots of specific details from \
its context. If the AI does not know the answer to a question, \
it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.
"Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?"

"""

In [ ]:
conversation.predict(input="What is 1+1?")

"""
> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. \
The AI is talkative and provides lots of specific details from its \
context. If the AI does not know the answer to a question, it \
truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew, it's nice to meet you. My name is AI. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.
'The answer to 1+1 is 2.'
"""

In [ ]:
conversation.predict(input="What is my name?")

"""


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an \
AI. The AI is talkative and provides lots of specific details from its \
context. If the AI does not know the answer to a question, it \
truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: The answer to 1+1 is 2.
Human: What is my name?
AI:

> Finished chain.
"I'm sorry, I don't have access to that information. Could you please tell me your name?"
"""

## 3. ConversationTokenBufferMemory

- This will limit the number of tokens saved

- Since many LLMs charge based on tokens this maps directly
  the const of LLM calls

### Example 3.1: A chatbot using `langchain`'s `ConversationTokenBufferMemory`

In [ ]:
#!pip install tiktoken

In [ ]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0)

In [ ]:
# llm is required to create ConversationTokenBufferMemory becase different
# llm's count tokens differently
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [ ]:
memory.load_memory_variables({})

"""
{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}
"""

## 4. ConversationSummaryMemory

- Here istead of limiting the stored conversation, we use an LLM
  to summarize the whole conversation and store that

### Example 4.1: A chatbot using `langchain`'s `ConversationSummaryBufferMemory`

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory


In [ ]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [ ]:
memory.load_memory_variables({})

"""
{'history': "System: The human and AI engage in small talk before discussing \
            the day's schedule. The AI informs the human of a morning meeting \
            with the product team, time to work on the LangChain project, and \
            a lunch meeting with a customer interested in the latest AI developments."}
"""

In [ ]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [ ]:
conversation.predict(input="What would be a good demo to show?")

"""
> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. \
The AI is talkative and provides lots of specific details from its \
context. If the AI does not know the answer to a question, it truthfully \
says it does not know.

Current conversation:
System: The human and AI engage in small talk before discussing the day's \
schedule. The AI informs the human of a morning meeting with the product team, \
time to work on the LangChain project, and a lunch meeting with a customer \
interested in the latest AI developments.
Human: What would be a good demo to show?
AI:

> Finished chain.
"Based on the customer's interest in AI developments, I would suggest showcasing \
our latest natural language processing capabilities. We could demonstrate how our AI \
can accurately understand and respond to complex language queries, and even provide \
personalized recommendations based on the user's preferences. Additionally, we could \
highlight our AI's ability to learn and adapt over time, making it a valuable tool for \
businesses looking to improve their customer experience."

"""

In [ ]:
memory.load_memory_variables({})

# Note: memory stores recent conversation upto the maximum tokens.
# Everything else is incorporated into the summary.

"""
{'history': "System: The human and AI engage in small talk before discussing \
            the day's schedule. The AI informs the human of a morning meeting \
            with the product team, time to work on the LangChain project, and \
            a lunch meeting with a customer interested in the latest AI \
            developments. The human asks what would be a good demo to show.\nAI: \
            Based on the customer's interest in AI developments, I would suggest \
            showcasing our latest natural language processing capabilities. We \
            could demonstrate how our AI can accurately understand and respond to \
            complex language queries, and even provide personalized recommendations \
            based on the user's preferences. Additionally, we could highlight our \
            AI's ability to learn and adapt over time, making it a valuable tool for \
            businesses looking to improve their customer experience."
}
"""

## Additional Memory Types

![Additional Memories](../images/L2/additional_memory_types.png)

- **Vector Data Memory** stores **word embeddings** and **vector embeddings**.

- **Entity Memory** can be use to remember facts about an **entity**, say a person.